In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19

content = r'dataset/content/lena.jpg'
style = r'dataset/style/mondrian.jpg'


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import os

# Function to load and preprocess image
def load_and_process_img(img_path, img_size=(512, 512)):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, img_size)
    img = img[tf.newaxis, :]
    return img

# Function to convert processed image to PIL image
def deprocess_img(processed_img):
    x = processed_img[0]
    x = np.clip(x, 0, 1)
    x = (x * 255).astype('uint8')
    return Image.fromarray(x)

# Paths to content and style images
content_path = r'dataset/content/lena.jpg'
style_path = r'dataset/style/mondrian.jpg'

# Load and preprocess images
content_image = load_and_process_img(content_path)
style_image = load_and_process_img(style_path)

# Load the style transfer model from TensorFlow Hub
print("🔄 Loading style transfer model from TensorFlow Hub...")
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
print("✅ Model loaded.")

# Run style transfer
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

# Save the output stylized image
output_image = deprocess_img(stylized_image.numpy())
output_image.save("output_stylized.jpg")
print("📸 Stylized image saved as 'output_stylized.jpg'.")

# Wrap TF Hub model into a Keras-style model to allow saving
class StyleTransferModel(tf.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, None, None, 3], dtype=tf.float32),
        tf.TensorSpec(shape=[None, None, None, 3], dtype=tf.float32)
    ])
    def __call__(self, content, style):
        return self.model(content, style)

# Save the model
style_transfer = StyleTransferModel(hub_model)
tf.saved_model.save(style_transfer, 'saved_model/style_transfer_model')
print("💾 Model saved to 'saved_model/style_transfer_model'.")
# Load the saved style transfer model
loaded_model = tf.saved_model.load('saved_model/style_transfer_model')

# Apply the model
stylized_output = loaded_model(tf.constant(content_image), tf.constant(style_image))[0]
# Save the model inside the Flask project model folder
MODEL_DIR = 'model/style_transfer_model'
tf.saved_model.save(style_transfer, MODEL_DIR)
print(f"💾 Model saved to '{MODEL_DIR}'.")
os.makedirs('model', exist_ok=True)


🔄 Loading style transfer model from TensorFlow Hub...
✅ Model loaded.
📸 Stylized image saved as 'output_stylized.jpg'.
INFO:tensorflow:Assets written to: saved_model/style_transfer_model\assets


INFO:tensorflow:Assets written to: saved_model/style_transfer_model\assets


💾 Model saved to 'saved_model/style_transfer_model'.
INFO:tensorflow:Assets written to: model/style_transfer_model\assets


INFO:tensorflow:Assets written to: model/style_transfer_model\assets


💾 Model saved to 'model/style_transfer_model'.
